# Семинар 8: ООП

Объектно ориентированное программирование на самом деле это по факту 4 принципа:
* **Abstraction** -- создавания классы мы меняем уровень абстракции, моделируем поведение класса и логику взаимодестсвия сущностей.
* **Наследование** -- нет смысла дублировать код, если можно выделить его в общего родителя для классов. Таким образом мы создаем новые абстракции на основе существующих.
* **Encapsulation** -- инкапсуляция, иными словами, скрытие внутри класса того, что не нужно явно использовать при работе с ним. Оставляем только публичный интерфейс работы с ним.
* **Polymorphism** -- разные классы могут реализовать разную логику одного и того же интерфейса.

Давайте напишем, например, свой класс комплексных чисел:

In [28]:
from typing import Union


class Complex:
    def __init__(self, re: Union[int, float] = 0, im: Union[int, float] = 0):
        self.re = re
        self.im = im
        
    def __str__(self) -> str:
        if not self.re and not self.im:
            return "0"

        str_re = str(self.re) if self.re else ""
        str_im = str(abs(self.im)) if abs(self.im) not in (0, 1) else ""
        i_sign = "i" if self.im else ""

        if self.im < 0:
            operator = "-"
        elif self.im > 0 and self.re:
            operator = "+"
        else:
            operator = ""

        return str_re + operator + str_im + i_sign

    __repr__ = __str__

In [29]:
a = Complex(0, 5)
b = Complex(8, -1)

Теперь добавим возможность складывать наши числа и умножать:

In [89]:
class Complex:
    def __init__(self, re: Union[int, float] = 0, im: Union[int, float] = 0):
        self.re = re
        self.im = im
        
    def __str__(self) -> str:
        if not self.re and not self.im:
            return "0"

        str_re = str(self.re) if self.re else ""
        str_im = str(abs(self.im)) if abs(self.im) not in (0, 1) else ""
        i_sign = "i" if self.im else ""

        if self.im < 0:
            operator = "-"
        elif self.im > 0 and self.re:
            operator = "+"
        else:
            operator = ""

        return str_re + operator + str_im + i_sign
    
    __repr__ = __str__  # можно и свой repr написать, а можно и так

    def __add__(self, other: Union[Complex, int, float]) -> Complex:  # <--- обратите внимание на кавычки
        # с Python 3.11 можно будет вместо "Complex" писать просто Self
        if isinstance(other, int) or isinstance(other, float):
            other = self.__class__(other, 0)

        return self.__class__(self.re + other.re, self.im + other.im)

    def __sub__(self, other: Complex) -> Complex:  # <--- обратите внимание на кавычки
        if isinstance(other, int) or isinstance(other, float):
            other = self.__class__(other, 0)
        return self.__class__(self.re - other.re, self.im - other.im)

    def __mul__(self, other: Complex) -> Complex:  # <--- обратите внимание на кавычки
        if isinstance(other, int) or isinstance(other, float):
            other = self.__class__(other, 0)
        return self.__class__(
            self.re * other.re - self.im * other.im,
            self.re * other.im + self.im * other.re
        )

    def __eq__(self, other: Complex) -> bool:
        return self.re == other.re and self.im == other.im

    def __neg__(self):
        return self.__class__(-self.re, -self.im)

    __radd__ = __add__
    __rmul__ = __mul__
    __rsub__ = __sub__

    # аналогично можно использовать __sub__ для вычитания, __div__ для деления

In [90]:
a = Complex(1, 5)
b = Complex(1, 5)

5 + a  # a + 5 -> radd

6+5i

In [91]:
-a

-1-5i

### Наследование

По факту, наследованием мы расширяем наш класс, создавания от него потомка, например:

In [110]:
import math

class Point(Complex):
    def length(self):
        return math.sqrt(self.re ** 2 + self.im ** 2)

In [111]:
x = Point(5, 6)
y = Point(-1, 1)

print(x.length())
print(y.length())

print((x + y).length())  # а вот это выполнится? если нет, то как поправить?

7.810249675906654
1.4142135623730951
8.06225774829855


In [82]:
import math

class Point(Complex):
    def distance(self, other: Point) -> float:
        # YOUR CODE HERE
        pass

### Задание 1

Немного "причешем" наше решение: давайте научим наш класс складываться и умножаться не только с комлексными числами, но и обычными

Для того, чтобы понять, какой это тип, можно использовать функцию `isinstance`.

### Задание 2

Давайте допишем класс Point, чтобы у него был метод distance, вычисляющий расстояние от нашей точки для другой. Как это лучше сделать?

### Ошибки

В питоне можно явно вызывать любую ошибку через raise, например:

In [105]:
import traceback

x = input()
y = int(input())

try:
    print(x / y)
except ZeroDivisionError:
    print("Division by zero occured")
except Exception:
    print("Some error idk what happened")
    print(traceback.format_exc())
finally:
    print("goodbye!")


Some error idk what happened
Traceback (most recent call last):
  File "/var/folders/r4/q_8q00yn5mx9rdsbtxdf4ghn0kx92w/T/ipykernel_90965/1189987990.py", line 7, in <module>
    print(x / y)
TypeError: unsupported operand type(s) for /: 'str' and 'int'

goodbye!


In [92]:
raise RuntimeError("Something went wrong")

RuntimeError: Something went wrong

А также можно создавать свои ошибки:

In [93]:
class ComplexError(BaseException):
    ...

raise ComplexError("Invalid usage of complex numbers")

ComplexError: Invalid usage of complex numbers

In [108]:
class ComplexOperationError(BaseException):
    def __init__(self, left_arg, right_arg):
        self.left_arg = left_arg
        self.right_arg = right_arg

    def __str__(self) -> str:
        return f"Cannot do operation between {self.left_arg} and {self.right_arg}"

# raise ComplexOperationError(Complex(1, 2), "abc")

### Задание 3

Допишите класс Complex таким образом, чтобы он возвращал ошибки, если мы пытаемся сложить/умножить, например, что-то невалидное (комплексное со строкой, например)

In [115]:
class Complex:
    def __init__(self, re: Union[int, float] = 0, im: Union[int, float] = 0):
        self.re = re
        self.im = im
        
    def __str__(self) -> str:
        if not self.re and not self.im:
            return "0"

        str_re = str(self.re) if self.re else ""
        str_im = str(abs(self.im)) if abs(self.im) not in (0, 1) else ""
        i_sign = "i" if self.im else ""

        if self.im < 0:
            operator = "-"
        elif self.im > 0 and self.re:
            operator = "+"
        else:
            operator = ""

        return str_re + operator + str_im + i_sign
    
    __repr__ = __str__  # можно и свой repr написать, а можно и так

    def __add__(self, other: Union[Complex, int, float]) -> Complex:  # <--- обратите внимание на кавычки
        # с Python 3.11 можно будет вместо "Complex" писать просто Self
        if isinstance(other, int) or isinstance(other, float):
            other = self.__class__(other, 0)
        elif not isinstance(other, Complex):
            raise ComplexOperationError(self, other)

        return self.__class__(self.re + other.re, self.im + other.im)

    def __sub__(self, other: Complex) -> Complex:  # <--- обратите внимание на кавычки
        if isinstance(other, int) or isinstance(other, float):
            other = self.__class__(other, 0)
        elif not isinstance(other, Complex):
            raise ComplexOperationError(self, other)
        return self.__class__(self.re - other.re, self.im - other.im)

    def __mul__(self, other: Complex) -> Complex:  # <--- обратите внимание на кавычки
        if isinstance(other, int) or isinstance(other, float):
            other = self.__class__(other, 0)
        elif not isinstance(other, Complex):
            raise ComplexOperationError(self, other)
        return self.__class__(
            self.re * other.re - self.im * other.im,
            self.re * other.im + self.im * other.re
        )

    def __eq__(self, other: Complex) -> bool:
        return self.re == other.re and self.im == other.im

    def __neg__(self):
        return self.__class__(-self.re, -self.im)

    __radd__ = __add__
    __rmul__ = __mul__
    __rsub__ = __sub__

    # аналогично можно использовать __sub__ для вычитания, __div__ для деления

In [120]:
a = Complex(5, 3)
b = Complex(-1, -2)

a + [1, 2, 3]

ComplexOperationError: Cannot do operation between 5+3i and [1, 2, 3]